### Workflow

1. Per la prima stringa si suggeriscono solo le skill che iniziano per la stringa inserita fino a quel momento.\
TODO: decidere quali mostrare, quelle più lunghe? a caso?


2. Per le successive stringe si considerano le skill inserite fino a quel momento: partendo dalla stringa inserita si ricavano le skill che contengono quella stringa, come in 1, poi per ognuna di queste si calcola la similarità del coseno con le skill già inserite e si suggerisce la skill con la media delle similarità più altra\
TODO: decidere quante skill mostrare. Usare la gerarchia delle skill come altra metrica di suggerimento, ad esempio in base alla distanza dal primo antenato in comune


3. suggerire anche una skill che non contenga la stringa inserita fino a quel momento ma che sia semplicemente la più simile con quelle inserite precedentemente, calcolata come per 2


4. Gestire gli errori di typo: individuarli e suggerire un'alternativa

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import Layout, Textarea, HBox, VBox, Label, Button
from gensim.models import FastText
from IPython.core.display import display

In [2]:
class FormUI:
        
    def __init__(self):
        
        #layout button
        self.layout_button = Layout(width='25%', height='50px')
        self.layout_textArea = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
        self.layout_buttonBox = Layout(flex='0 1 auto', height='100px', width='90%')
        
        # lista dei bottoni
        self.suggest_buttons = list()
        self.suggest_buttons_context = list()
        
        # set up della text area
        self.textArea = Textarea(
            value='',
            placeholder='Type something',
            disabled=False,
            tooltip='Enter the name of the Text field',
            height='90px',
            layout=self.layout_textArea
        )
        
        # set up della box per i bottoni dei suggerimenti per l'input
        self.suggests = HBox(self.suggest_buttons, layout=self.layout_buttonBox)
        
        # set up della box per i bottoni dei suggerimenti solo per il context
        self.suggests_context = HBox(self.suggest_buttons_context, layout=self.layout_buttonBox)
        
        
        # set up dell'intero form
        self.form = VBox([VBox([Label(value='Skills:'), self.textArea]), 
                          HBox([Label(value='Suggests:'), self.suggests]),
                          HBox([Label(value='Similar skills:'), self.suggests_context])])
        
    def show_form(self):
        display(self.form)
        
        
    
    def init_button(self, description, button_style, tooltip):

        b = Button(
            description=description,
            disabled=False,
            button_style=button_style,  # 'success', 'info', 'warning', 'danger' or ''
            tooltip=tooltip,
            layout=self.layout_button
        )

        return b
    
    
    def close_suggest_buttons(self):
        '''
        Remove button of old suggestions
        '''
        for w in self.suggest_buttons:
            w.close()
        self.suggest_buttons = list()
    
    
    def close_suggest_buttons_context(self):
        '''
        Remove button of old suggestions
        '''
        for w in self.suggest_buttons_context:
            w.close()
        self.suggest_buttons_context = list()
   
    

In [6]:
class AutoCompleteManager:
    
    def __init__(self, skills_list, model, cos_sim_matrix, alpha=0.8, num_suggests=4):
        # peso dell'input inserito nel calcolo della similarità finale
        self.alpha = alpha
        self.num_suggests = num_suggests
        
        self.skills_list = skills_list
        self.model = model
        self.cos_sim_matrix = cos_sim_matrix
        
        self.FormUI = FormUI()
    
    def get_skills_input_similarity(self, word, context):
        '''
        Find all skill that contain the new string insert by user
        '''
        return {k: self.model.wv.similarity(word, k) for k in self.skills_list if k not in context}
        #return [w for w in sorted(words_list) if w.lower().startswith(word.lower())]
        
    def get_skills_context_similarity(self, context):
        res = dict()
        for s in self.skills_list:
            if s not in context:
                res[s] = np.array([self.model.wv.similarity(s, c) for c in context]).mean()
        return res
    
    
    def get_best_similarity_skill(self, new_input, context):
        '''
        Get the four most similarity skills respect of the user's input
        '''
        # similarità tra input e lista delle skills
        similarity_with_input = self.get_skills_input_similarity(new_input, context)
        res_sim = dict()
        
        # calcolo delle medie delle similarità tra le skills candidate come suggerimenti e 
        # le skills già inserite
        similarity_with_context = self.get_skills_context_similarity(context)
        
        # calcolo della similarità complessiva
        for skill in similarity_with_context.keys():
            res_sim[skill] = self.alpha*similarity_with_input[skill]+(1-self.alpha)*similarity_with_context[skill]
            
        return {k: v for k, v in sorted(res_sim.items(), key=lambda item: -item[1])[:self.num_suggests]}
        #(a * sim(parola_corrente, parole_excel) + (1-a) * media(similarità(parola_precedente, parole, excel)))
        #candidate_skill = self.cos_sim_matrix.loc[context, find_suggests]
        #return candidate_skill.mean(axis = 0).sort_values(ascending=False)
        #return candidate_skill.mean(axis = 0).idxmax(), candidate_skill.mean(axis = 0).max()
        
        
    def suggest_interest_skill(self, context):
        #ll = list()
        #for word in [w for w in self.cos_sim_matrix.columns if w not in context]:
         #   ll.append((word, self.cos_sim_matrix.loc[word, context].mean()))
        
        similarity_context = self.get_skills_context_similarity(context)
        similarity_context = {k: v for k, v in sorted(similarity_context.items(), 
                                                      key=lambda item: -item[1])[:self.num_suggests]}
        
        for index, value in similarity_context.items():
            b = self.FormUI.init_button(description=f'{index} - \n{round(value * 100, 2)}%',
                                        button_style='info',
                                        tooltip=f'{index} - {round(value * 100, 2)}%')
            
            b.on_click(self.add_skill)
            self.FormUI.suggest_buttons_context.append(b)
            
        self.FormUI.suggests_context.children=tuple(self.FormUI.suggest_buttons_context)
    
    
    def add_skill(self, btn_object):
        '''
        Button click handler, add the skill pressed to the text area
        '''
        skill_list = self.FormUI.textArea.value.split(', ')
        skill_list[-1] = btn_object.description.split(" -")[0]
        new_contest = ', '.join(skill_list) + ', '
        self.FormUI.textArea.value = new_contest
        self.FormUI.close_suggest_buttons()
        self.FormUI.close_suggest_buttons_context()
        self.suggest_interest_skill(skill_list)
        
        
    def suggests_manager(self, widget):
        '''
        Main function of the class, manage the suggestions in different case
        '''
        self.FormUI.close_suggest_buttons()
        old_input = widget['old']
        new_input = widget['new'].split(' ')[-1]
        context = old_input.split(', ')[:-1]
        if len(new_input)>3:
            if context == []:
                similarity_input = self.get_skills_input_similarity(new_input, context)
                similarity_input = dict(sorted(similarity_input.items(), 
                                                    key=lambda x: -x[1])[:self.num_suggests])

                for index, value in similarity_input.items():
                    b = self.FormUI.init_button(description=f'{index} - \n{round(value * 100, 2)}%',
                                                button_style='success',
                                                tooltip=f'{index} - {round(value * 100, 2)}%')

                    b.on_click(self.add_skill)
                    self.FormUI.suggest_buttons.append(b)
            else:
                best_similarity = self.get_best_similarity_skill(new_input, context)
                
                for index, value in best_similarity.items():
                    b = self.FormUI.init_button(description=f'{index} - \n{round(value * 100, 2)}%',
                                                button_style='success',
                                                tooltip=f'{index} - {round(value * 100, 2)}%')
                    b.on_click(self.add_skill)
                    self.FormUI.suggest_buttons.append(b)

                #display(widgets.HBox(self.suggest_buttons))
        self.FormUI.suggests.children=tuple(self.FormUI.suggest_buttons)
        
    
    def show_form(self):
        '''
        Show the text area widget
        '''
        self.FormUI.show_form()
        self.FormUI.textArea.observe(self.suggests_manager, names='value')
    

In [4]:
skills_list = pd.read_excel("data/2020_06_09 Allocation to ONET.xlsx")['escoskill_level_3']
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")
skills_vectors = {k: vectors.wv[k] for k in skills_list}
cos_sim_matrix = pd.DataFrame(cosine_similarity(np.array(list(skills_vectors.values()))),
                                   columns=list(skills_vectors.keys()),
                                   index=list(skills_vectors.keys()))

/home/simone/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  


In [ ]:
t = AutoCompleteManager(skills_list, vectors, cos_sim_matrix, alpha=0.8, num_suggests=4)

t.show_form()